In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#from tensorflow.python.client import device_lib
#print device_lib.list_local_devices()

In [2]:
import tensorflow as tf
import numpy as np
from pdrnn import triangular_pmf, build_pdrnn, compute_loss
from StochasticDilateNet import StochasticDilateNet
from scipy.stats import bernoulli

In [3]:
tf.reset_default_graph()
hidden_structs = [4]*2
init_params = [(1.0,0.5),(100.0,50.0)]
model = StochasticDilateNet(hidden_structs,
                            init_params,
                            n_layers=2,
                            n_classes=2,
                            input_dims=1,
                            lambda_b=0.9,
                            cell_type="RNN")
# define session
sess_config = tf.ConfigProto(allow_soft_placement=True,
                             log_device_placement=False,
                             intra_op_parallelism_threads=20,
                             inter_op_parallelism_threads=4)
sess_config.gpu_options.allow_growth = True

In [4]:
with tf.Session(config=sess_config) as sess:
    # initialize all the parameters
    sess.run(tf.global_variables_initializer())
    
    for step in range(1000):
        seq = bernoulli.rvs(0.5, size=(1000,256,1))
        feed_dict = {model.inputs: seq,
                     model.labels: seq[0,:,0]}
        _, loss_value = sess.run([model.weights_train_op, model.loss_for_w], 
                                       feed_dict=feed_dict)
        assert not np.isnan(loss_value), 'Model diverged with loss = NaN'
        
        
        sess.run(model.struct_train_op,
                       feed_dict=feed_dict)
        
        struct_params, rates, b, accuracy = sess.run([model.struct_vars, model.rates, model.b, model.accuracy],
                                                 feed_dict=feed_dict)
        print('Before Clipping {}, Rates {}, b = {}, accuracy = {}'.format(struct_params, rates, b, accuracy))
        
        sess.run(model.struct_clip_op, feed_dict={model.inputs: seq})
        
        struct_params = sess.run(model.struct_vars)
        print('Step {}, struct params {}, loss = {}'.format(step, struct_params, loss_value))

Before Clipping [(1.0, 0.5), (99.9, 49.900005)], Rates [1, 118], b = 0.295628666878, accuracy = 0.51171875
Step 0, struct params [(2.0, 1.5), (99.9, 49.900005)], loss = 1.60100984573
Before Clipping [(2.0, 1.5), (99.889656, 49.809036)], Rates [1, 130], b = 0.53723025322, accuracy = 0.51171875
Step 1, struct params [(2.0, 1.5), (99.889656, 49.809036)], loss = 1.61814165115
Before Clipping [(2.0, 1.5), (99.919235, 49.718998)], Rates [2, 102], b = 0.712356567383, accuracy = 0.53515625
Step 2, struct params [(2.0, 1.5), (99.919235, 49.718998)], loss = 1.43757724762
Before Clipping [(2.0, 1.5), (99.92398, 49.63707)], Rates [2, 87], b = 0.878156423569, accuracy = 0.4765625
Step 3, struct params [(2.0, 1.5), (99.92398, 49.63707)], loss = 1.60265910625
Before Clipping [(2.0, 1.5), (99.9395, 49.55741)], Rates [1, 97], b = 0.970369398594, accuracy = 0.56640625
Step 4, struct params [(2.0, 1.5), (99.9395, 49.55741)], loss = 1.38835930824
Before Clipping [(2.0, 1.5), (99.97043, 49.47385)], Rates [

Before Clipping [(2.0, 1.5), (100.31246, 48.754677)], Rates [1, 96], b = 0.950990080833, accuracy = 0.578125
Step 44, struct params [(2.0, 1.5), (100.31246, 48.754677)], loss = 0.978441238403
Before Clipping [(2.0, 1.5), (100.31412, 48.761013)], Rates [1, 118], b = 0.960390925407, accuracy = 0.4609375
Step 45, struct params [(2.0, 1.5), (100.31412, 48.761013)], loss = 0.946476578712
Before Clipping [(2.0, 1.5), (100.314575, 48.766636)], Rates [2, 98], b = 0.964907705784, accuracy = 0.5
Step 46, struct params [(2.0, 1.5), (100.314575, 48.766636)], loss = 0.952897787094
Before Clipping [(2.0, 1.5), (100.31481, 48.77177)], Rates [2, 133], b = 0.969757974148, accuracy = 0.48046875
Step 47, struct params [(2.0, 1.5), (100.31481, 48.77177)], loss = 1.01354491711
Before Clipping [(2.0, 1.5), (100.31527, 48.776314)], Rates [2, 116], b = 0.97336268425, accuracy = 0.4609375
Step 48, struct params [(2.0, 1.5), (100.31527, 48.776314)], loss = 1.02085638046
Before Clipping [(2.0, 1.5), (100.31604, 

KeyboardInterrupt: 

In [ ]:
print(model.sess.run(tf.gradients(model.loss_for_pi, model.struct_vars[0]), feed_dict = feed_dict))

In [ ]:
print model.sess.run(model.loss_per_example, feed_dict=feed_dict)

In [ ]:
# build the network
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, shape = [None, None, 1], name = 'inputs')

logits, structs, rates = build_pdrnn(x=inputs,
                                     hidden_structs=[4]*3,
                                     init_params=[(2.0,2.0),(2.0,2.0),(2.0,2.0)],
                                     n_layers=3,
                                     n_steps=20,
                                     n_classes=2,
                                     input_dims=1,
                                     cell_type="RNN")

In [ ]:
a = tf.ones([10, 16])
b = tf.split(a, [1]*10)

print(b)

In [ ]:
t = tf.constant([[1, 2, 3], [4, 5, 6]])
paddings = tf.constant([[0, 0], [0, 3]])
with tf.Session():
    print tf.pad(t, paddings, constant_values=0.0).eval() 

In [31]:
tf.reset_default_graph()
init_params=[(1.0,0.5),(2.0,2.0)]
dilations = []
for l in xrange(2):
    with tf.variable_scope('struct_layer_{}'.format(l)):
        mu = tf.get_variable('mu', initializer=tf.constant(init_params[l][0]))
        sigma = tf.get_variable('sigma', initializer=tf.constant(init_params[l][1]))
                
    rates = tf.range(tf.ceil(tf.maximum(mu-sigma,0.5)), 
                     tf.ceil(tf.minimum(mu+sigma,20)))
    probs = triangular_pmf(rates, mu, sigma, 20)
    probs = tf.expand_dims(probs, 0)
    idx = tf.multinomial(tf.log(probs), 1)
    rates = tf.cast(rates, tf.int32)
    dilations.append(rates[idx[0][0]])  

In [29]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print sess.run(probs)

[[1.]]


In [33]:
a=[v for v in tf.trainable_variables() if 'struct_layer' in v.name]
a[2:]

[<tf.Variable 'struct_layer_1/mu:0' shape=() dtype=float32_ref>,
 <tf.Variable 'struct_layer_1/sigma:0' shape=() dtype=float32_ref>]

In [ ]:
from classification_models import drnn_classification, _contruct_cells
from models import dRNN, multi_dRNN_with_dilations
# build the network
tf.reset_default_graph()

dilations = []
for l in xrange(3):
    d = tf.placeholder_with_default(tf.constant(l+1), [])
    dilations.append(d)

inputs = tf.placeholder(tf.float32, shape = [None, None, 1], name = 'inputs')
hidden_structs=[4]*3

# construct a list of cells
cells = _contruct_cells(hidden_structs=hidden_structs, cell_type='LSTM')

logits = multi_dRNN_with_dilations(cells, inputs, hidden_structs, dilations)
#logits = dRNN(cells[0], inputs, dilations[0], 1, scope='default')

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    pred = sess.run(logits, feed_dict={inputs: np.random.uniform(size=[25,33,1])})
    print pred.shape